In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os
import prepare
import wrangle
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import explore
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from math import sqrt
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
def get_auto_mpg():
    
    '''Acquire, clean, and return the auto-mpg dataset'''
    
    df = pd.read_fwf('auto-mpg.data', header=None)
    
    df.columns = ['mpg', 'cylinders', 'displ', 'horsepower', 'weight', 'acc',
              'model_year', 'origin', 'name']
    
    df = df[df['horsepower'] != '?']
    
    df['horsepower'] = df['horsepower'].astype('float')
    
    return df

In [3]:
df = get_auto_mpg()
df.head()

,mpg,cylinders,displ,horsepower,weight,acc,model_year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,"""ford torino"""


In [4]:
train, val, test = wrangle.train_val_test(df)

In [5]:
mms = MinMaxScaler()

train[['displ','horsepower','weight','acc']] = mms.fit_transform(train[['displ', 'horsepower', 'weight','acc']])
train.head()

,mpg,cylinders,displ,horsepower,weight,acc,model_year,origin,name
216,31.5,4,0.077519,0.122905,0.118280,0.625000,77,3,"""honda accord cvcc"""
348,37.7,4,0.054264,0.089385,0.119773,0.553571,81,3,"""toyota tercel"""
232,16.0,8,0.731266,0.575419,0.802270,0.386905,77,1,"""ford thunderbird"""
201,18.5,6,0.470284,0.357542,0.596177,0.488095,76,1,"""pontiac ventura sj"""
102,26.0,4,0.074935,0.000000,0.089904,0.773810,73,2,"""volkswagen super beetle"""


## Select K best


In [6]:
x_train_scaled = train[['displ', 'horsepower', 'weight','acc']]
y_train = train['mpg']

In [7]:
f_selector = SelectKBest(f_regression, k = 2)

In [8]:
f_selector.fit(x_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x12c700f70>)

In [9]:
f_select_mask = f_selector.get_support()

In [10]:
x_train_scaled.columns[f_select_mask]

Index(['displ', 'weight'], dtype='object')

## RFE

In [11]:
x_train = train.drop(columns = ['mpg', 'model_year','name'])

In [12]:
x_train = pd.get_dummies(x_train, columns = ['cylinders', 'origin'])

In [13]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select = 7)

In [14]:
rfe.fit(x_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=7)

In [15]:
ranks = rfe.ranking_
columns = x_train.columns.tolist()

In [16]:
feature_ranks = pd.DataFrame({'ranking':ranks,'feature': columns})

In [17]:
feature_ranks.sort_values('ranking')

,ranking,feature
0,1,displ
1,1,horsepower
2,1,weight
4,1,cylinders_3
6,1,cylinders_5
7,1,cylinders_6
11,1,origin_3
3,2,acc
5,3,cylinders_4
8,4,cylinders_8


# regression model


In [25]:
x_train.drop(columns=['cylinders_5'], inplace = True)

In [26]:
from sklearn.metrics import mean_squared_error
val[['displ', 'horsepower', 'weight', 'acc']] = mms.transform(val[['displ', 'horsepower', 'weight', 'acc']])
x_val = val.drop(columns=['mpg', 'model_year', 'name'])
x_val = pd.get_dummies(x_val, columns=['cylinders', 'origin'])
y_val = val['mpg']

In [27]:
from sklearn.linear_model import LinearRegression

In [28]:
lm = LinearRegression()

In [29]:
lm.fit(x_train, y_train)

LinearRegression()

In [30]:
lm_preds = lm.predict(x_train)

In [31]:
preds_df = pd.DataFrame({'actual': y_train, 'lm_preds': lm_preds})

In [32]:
from math import sqrt

In [33]:
sqrt(mean_squared_error(preds_df['lm_preds'], preds_df['actual']))

3.614447872493064

# Lasso + Lars

In [34]:
from sklearn.linear_model import LassoLars

In [43]:
lasso = LassoLars(alpha = 0.15)

In [44]:
lasso.fit(x_train, y_train)

LassoLars(alpha=0.15)

In [45]:
lasso_preds = lasso.predict(x_train)

In [46]:
preds_df['lasso_preds'] = lasso_preds
preds_df.head()

,actual,lm_preds,lasso_preds
216,31.5,32.040603,27.484862
348,37.7,32.709007,27.565715
232,16.0,14.622506,17.236151
201,18.5,17.507690,20.236920
102,26.0,30.553219,27.872163


In [48]:
sqrt(mean_squared_error(preds_df['actual'], preds_df['lasso_preds']))

4.740402854114932

# polynomial regression 

for non linear relationships

In [49]:
from sklearn.preprocessing import PolynomialFeatures

In [50]:
pf = PolynomialFeatures(degree = 2)

In [51]:
pf.fit(x_train, y_train)

PolynomialFeatures()

In [52]:
pf.transform(x_train)

array([[1.        , 0.07751938, 0.12290503, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.05426357, 0.08938547, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.73126615, 0.57541899, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.6124031 , 0.52513966, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.47028424, 0.23463687, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.00775194, 0.10614525, ..., 0.        , 0.        ,
        1.        ]])